In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import os
import scipy.io as io 
import numpy as np
import glob
import pandas as pd

In [2]:
os.chdir('/home/raghuram/Desktop/radiomics/TEXTURES')
!pwd
!ls *.mat| wc -l

/home/raghuram/Desktop/radiomics/TEXTURES
408


In [3]:
import scipy
import numpy as np
import scipy.io as spio


def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [4]:
expt_list = ['Experiment'+str(i) for i in range(1,41)]

In [5]:
T1CE_set = set(glob.glob('*_T1CE.mat'))
T2W_set = set(glob.glob('*_T2W.mat'))
T1W_set = set(glob.glob('*_T1W.mat'))
T2F_set = set(glob.glob('*_T2F.mat'))

In [6]:
import collections

In [8]:
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [9]:
df = pd.read_csv('/home/raghuram/Desktop/radiomics/Patient_Information_and_Metadata.csv')

In [57]:
folder = '/home/raghuram/Desktop/radiomics/expt/data/T2F'

In [53]:
mag_pattern = 'Name: Magnetization Strength, dtype: float64'
scanner_pattern = 'Name: Scanner Manufacturer, dtype: object'
idh1_pattern = 'Name: IDH1, dtype: float64'
codel_pattern = 'Name: 1p_19q_co_del_status, dtype: float64'
death_pattern = 'Name: days_to_last_or_death, dtype: float64'

In [7]:
for idx, expt_ in enumerate(expt_list):
    if idx%10 == 0:
        print('Experiment {} finished'.format(idx))
    data = []
    for mat_file in T2F_set:
        info = loadmat(mat_file)
        print(info)
        break
        output = flatten(info['textures'][expt_])
        mag_field_value = str(df[df['T2F'] == mat_file]['Magnetization Strength']).replace(mag_pattern, '')
        scanner_name = str(df[df['T2F'] == mat_file]['Scanner Manufacturer']).replace(scanner_pattern, '')
        idh1_status = str(df[df['T2F'] == mat_file]['IDH1']).replace(idh1_pattern, '')
        p19q_status = str(df[df['T2F'] == mat_file]['1p_19q_co_del_status']).replace(codel_pattern, '')
        death_days = str(df[df['T2F'] == mat_file]['days_to_last_or_death']).replace(death_pattern, '')
        
        if idh1_status == 'Series([], )':
            idh1_status = None
        elif idh1_status is not None:
            idh1_status = float(str(idh1_status.split(' ')[-1]).strip())
        
        if p19q_status == 'Series([], )':
            p19q_status = None
        elif p19q_status is not None:
            p19q_status = float(str(p19q_status.split(' ')[-1]).strip())
        
        if death_days == 'Series([], )':
            death_days = None
        elif death_days is not None:
            death_days = float(str(death_days.split(' ')[-1]).strip())
            
        if mag_field_value == 'Series([], )':
            mag_field_value = None
        elif mag_field_value is not None:
            mag_field_value = mag_field_value.split(' ')[-1].strip()
        if scanner_name == 'Series([], )':
            scanner_name = None
        else:
#             print(scanner_name.split('   '))
            scanner_name = scanner_name.split('    ')[1].strip()
#             print(scanner_name.strip())
        output['Magnetic Strength'] = mag_field_value
        output['Scanner Manufacturer'] = scanner_name
        output['Filename'] = mat_file
        output['idh_status'] = idh1_status
        output['codel_status'] = p19q_status
        output['death_days'] = death_days
        data.append(output) 
        
    df_ = pd.DataFrame.from_dict(data)
    df_.to_csv(os.path.join(folder,'Features_'+expt_+'.csv'), index=False)

print ('Finished.')

Experiment 0 finished
{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sat Apr 30 02:53:47 2016', '__version__': '1.0', '__globals__': [], 'textures': {'List': {'Experiment1': 'Scale=0.5, Quant.algo=Equal, Ng=8', 'Experiment2': 'Scale=0.5, Quant.algo=Equal, Ng=16', 'Experiment3': 'Scale=0.5, Quant.algo=Equal, Ng=32', 'Experiment4': 'Scale=0.5, Quant.algo=Equal, Ng=64', 'Experiment5': 'Scale=0.5, Quant.algo=Uniform, Ng=8', 'Experiment6': 'Scale=0.5, Quant.algo=Uniform, Ng=16', 'Experiment7': 'Scale=0.5, Quant.algo=Uniform, Ng=32', 'Experiment8': 'Scale=0.5, Quant.algo=Uniform, Ng=64', 'Experiment9': 'Scale=1, Quant.algo=Equal, Ng=8', 'Experiment10': 'Scale=1, Quant.algo=Equal, Ng=16', 'Experiment11': 'Scale=1, Quant.algo=Equal, Ng=32', 'Experiment12': 'Scale=1, Quant.algo=Equal, Ng=64', 'Experiment13': 'Scale=1, Quant.algo=Uniform, Ng=8', 'Experiment14': 'Scale=1, Quant.algo=Uniform, Ng=16', 'Experiment15': 'Scale=1, Quant.algo=Uniform, Ng=32', 'Experiment16': 'Scale

NameError: name 'folder' is not defined